# Prepare Phase: New Business Location Predictor

In [1]:
import numpy as np
import pandas as pd

import acquire as a
import location as l
from env import api

In [2]:
sex = a.get_sex_age_data()
sex.head()

,geography,total population,sex ratio (males per 100 females),old-age dependency ratio,child dependency ratio
0,110100,3153,161.7,29.0,6.4
1,110300,3114,99.9,13.0,43.4
2,110500,2430,100.3,25.4,89.8
3,110600,5645,275.1,15.0,11.6
4,110700,1079,129.1,49.2,35.8


In [3]:
race = a.get_race_data()
race.head()

,geography,total_hispanic_or_latino
0,110100,1758
1,110300,1589
2,110500,1982
3,110600,4763
4,110700,933


In [4]:
income = a.get_income_data()
income.head()

,geography,households_median_income_(dollars)
0,110100,52659
1,110300,43875
2,110500,10518
3,110600,16712
4,110700,18700


In [5]:
# now merge this all into one df:
df = pd.merge(pd.merge(sex, race, on='geography'), income, on='geography')
df.head()

,geography,total population,sex ratio (males per 100 females),old-age dependency ratio,child dependency ratio,total_hispanic_or_latino,households_median_income_(dollars)
0,110100,3153,161.7,29.0,6.4,1758,52659
1,110300,3114,99.9,13.0,43.4,1589,43875
2,110500,2430,100.3,25.4,89.8,1982,10518
3,110600,5645,275.1,15.0,11.6,4763,16712
4,110700,1079,129.1,49.2,35.8,933,18700


In [6]:
# testing the get_census_data function:
census = a.get_census_data()
census.head()

,geography,total population,sex ratio (males per 100 females),old-age dependency ratio,child dependency ratio,total_hispanic_or_latino,households_median_income_(dollars)
0,110100,3153,161.7,29.0,6.4,1758,52659
1,110300,3114,99.9,13.0,43.4,1589,43875
2,110500,2430,100.3,25.4,89.8,1982,10518
3,110600,5645,275.1,15.0,11.6,4763,16712
4,110700,1079,129.1,49.2,35.8,933,18700


In [7]:
yoga_coords = l.get_bexar_yoga_studios(api)

In [8]:
yoga_coords

,latitude,longitude
0,29.479815,-98.492192
1,29.466683,-98.490809
2,29.614283,-98.312855
3,29.485545,-98.569729
4,29.727999,-98.094677
5,29.610592,-98.491604
6,29.792908,-98.729822
7,29.534977,-98.521877
8,29.521187,-98.709042
9,29.431522,-98.482394


In [9]:
l.assign_census_tract(yoga_coords)

,latitude,longitude,census_tract
0,29.479815,-98.492192,1207.01
1,29.466683,-98.490809,1207.01
2,29.614283,-98.312855,1207.01
3,29.485545,-98.569729,1207.01
4,29.727999,-98.094677,1207.01
5,29.610592,-98.491604,1207.01
6,29.792908,-98.729822,1207.01
7,29.534977,-98.521877,1207.01
8,29.521187,-98.709042,1207.01
9,29.431522,-98.482394,1207.01
